<a href="https://colab.research.google.com/github/Sambhavmahajan11/Blog-Site/blob/main/NLP_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Load one of the availables trained pipelines for English
# English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
nlp = spacy.load('en_core_web_sm')


# stop words built in spacy (english)
print(nlp.Defaults.stop_words)

{'anywhere', 'very', 're', 'then', 'another', 'neither', 'will', 'only', 'seemed', 'over', 'five', 'after', 'what', 'sometime', 'something', 'an', 'less', 'been', 'for', 'which', 'always', 'anyhow', 'whereby', 'these', 'that', 'whereupon', 'enough', 'take', 'as', 'somewhere', 'under', 'be', 'thru', 'again', 'could', 'behind', '’ll', 'perhaps', 'almost', 'whether', 'else', 'the', 'it', 'does', 'he', 'ten', 'say', 'hers', 'afterwards', 'mostly', 'became', 'while', 'herein', 'without', 'using', 'seeming', 'whom', 'someone', 'full', 'them', 'wherein', 'to', "'s", 'itself', 'give', 'whose', 'any', 'beyond', 'until', 'is', 'those', 'six', 'other', 'here', 'within', 'or', 'go', 'herself', 'back', 'many', 'least', 'her', 'namely', 'towards', 'along', 'see', 'via', 'further', 'mine', 'never', 'they', 'next', 'n‘t', 'anyway', 'you', 'hereupon', 'put', 'done', 'since', 'ever', 'by', 'three', '‘m', 'into', 'latter', 'sixty', 'also', "n't", 'otherwise', 'made', 'used', 'was', 'thereby', 'please', '

In [4]:
print(f"Number of default stop words : {len(nlp.Defaults.stop_words)}")

Number of default stop words : 326


In [5]:
s_stemmer = SnowballStemmer(language='english')

In [6]:
words = ['run', 'runner', 'ran', 'runs', 'easily', 'fairly', 'fairness','boats','boating']

In [7]:
for word in words:
    print(word+ ' ------> ' + s_stemmer.stem(word))

run ------> run
runner ------> runner
ran ------> ran
runs ------> run
easily ------> easili
fairly ------> fair
fairness ------> fair
boats ------> boat
boating ------> boat


In [8]:
# Function to display lemmas
def show_lemmas(text):
    for token in text:
        print(f'{token.text:{12}} {token.pos_:{6}} {token.lemma:<{22}} {token.lemma_}')

In [9]:
doc = nlp(u"I saw eighteen mice today!")
show_lemmas(doc)

I            PRON   561228191312463089     -PRON-
saw          VERB   11925638236994514241   see
eighteen     NUM    9609336664675087640    eighteen
mice         NOUN   1384165645700560590    mouse
today        NOUN   11042482332948150395   today
!            PUNCT  17494803046312582752   !


In [10]:
doc = nlp(u"I am meeting him tomorrow at the meeting.")
show_lemmas(doc)

I            PRON   561228191312463089     -PRON-
am           AUX    10382539506755952630   be
meeting      VERB   6880656908171229526    meet
him          PRON   561228191312463089     -PRON-
tomorrow     NOUN   3573583789758258062    tomorrow
at           ADP    11667289587015813222   at
the          DET    7425985699627899538    the
meeting      NOUN   14798207169164081740   meeting
.            PUNCT  12646065887601541794   .


In [11]:
count_vect = CountVectorizer()

In [12]:
phrase = ["I'd like to have a glass of water please"]

In [13]:
count_vect.fit(phrase)

CountVectorizer()

In [14]:
count_vect.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



['glass', 'have', 'like', 'of', 'please', 'to', 'water']

In [15]:
count_vect.fit_transform(phrase)

<1x7 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [16]:
count_vect.vocabulary_

{'glass': 0, 'have': 1, 'like': 2, 'of': 3, 'please': 4, 'to': 5, 'water': 6}

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MBTI 500.csv')

In [19]:
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ


In [20]:
df['posts'][0]

'know intj tool use interaction people excuse antisocial truly enlighten mastermind know would count pet peeze something time matter people either whether group people mall never see best friend sit outside conversation jsut listen want interject sit formulate say wait inject argument thought find fascinate sit watch people talk people fascinate sit class watch different people find intrigue dad intj u stand look like line safeway watch people home talk people like think military job people voluntarily go job important show deference endanger live glorify way civilian think pretty ignorant general think military necessary defense mechanism political tactic feel like u specifically invest much money could put money education whatnot though personally sound budget aernative really comment one way base two politician eye year ago come name somewhat important kinda role model nowadays pick keep score individual level mean little vary accord number condition day may score high others low sw

In [21]:
df['type'][0]

'INTJ'

In [22]:
df['type'].unique()

array(['INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP', 'ENFJ', 'ENFP',
       'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP'],
      dtype=object)

In [23]:
print(f"Total of {len(df['type'].unique())} types of classified MBTI posts")

Total of 16 types of classified MBTI posts


In [24]:
df.isnull().sum()

posts    0
type     0
dtype: int64

In [25]:
df_bar_chart=df.groupby('type').count()


trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['posts'])

data = [trace1]
layout = go.Layout(title='MBTI # Classified Posts per Type')

fig = go.Figure(data=data, layout=layout)

fig.show()

In [26]:
recreate_model=False

In [27]:
filename = 'mbti_svm_v2.sav'

In [28]:
if not os.path.isfile(filename):
    recreate_model=True

In [29]:
X = df['posts'] # features
y = df['type']  # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Check if need to recreate the model
if recreate_model:    
    
    # Creating an instance to vectorizer:
    vectorizer = TfidfVectorizer()
    
    # Training the vectorizer:
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Training the classifier:
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # Pipelining the vectorizer and the classifier
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # saving the model to disk
    pickle.dump(text_clf, open(filename, 'wb'))

# If there is no need to recreate the model, just open the file from the disk    
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

In [31]:
predictions = text_clf.predict(X_test)

In [32]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.84      0.58      0.69       319
        ENFP       0.82      0.78      0.80      1249
        ENTJ       0.90      0.80      0.84       577
        ENTP       0.86      0.83      0.84      2324
        ESFJ       0.83      0.45      0.59        33
        ESFP       0.88      0.48      0.62        75
        ESTJ       0.90      0.84      0.87       105
        ESTP       0.95      0.90      0.92       398
        INFJ       0.81      0.84      0.83      2954
        INFP       0.80      0.82      0.81      2391
        INTJ       0.83      0.87      0.85      4531
        INTP       0.84      0.87      0.86      5033
        ISFJ       0.80      0.61      0.69       132
        ISFP       0.81      0.60      0.69       161
        ISTJ       0.86      0.68      0.76       253
        ISTP       0.89      0.79      0.84       679

    accuracy                           0.84     21214
   macro avg       0.85   

In [33]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.84


In [52]:
var =text_clf.predict(["Hello world"])
str(var[0])


'INFP'

In [53]:
import anvil.server

anvil.server.connect("IWKPVKIJVQ6CRQJZOMOGHNCJ-IJH2S6DUEMXGP6AM")


In [54]:
@anvil.server.callable

def submit_button_click(txt):
    # Set 'name' to the text in the 'name_box'
    print(txt)
    type=text_clf.predict([txt])

    return str(type[0])

